In [1]:
%reload_ext autoreload
%load_ext dotenv 
%dotenv -o -v .env

from DbConnexion import DbConnexion



Db = DbConnexion(required_privileges={'SELECT'})

--- Initialisation de la connexion à la BDD ---
sae204now
Connexion à la base de données sae204now réussie !
--- vérification de la BDD ---
ping...
ping réussi
vérification des permissions...
Récupération des privilèges de l'utilisateur...
vérification des permissions réussie
Vérification des tables...
La table history_change existe
La table station_information existe
La table station_status existe


In [2]:
import folium
import mysql.connector
from mysql.connector import errorcode
from colors import Fore, Style
from cursorContext import cursorContext
import pandas as pd


query = """
SELECT i.stationcode, i.name, i.capacity, ST_X(coordonnees_geo) AS lon, ST_Y(coordonnees_geo) AS lat, 
i.nom_arrondissement_communes, s.date, s.is_installed, s.numdocksavailable, s.numbikesavailable, s.mechanical, s.ebike
    FROM station_information i 
    INNER JOIN station_status s ON s.stationcode = i.stationcode 
    WHERE s.date = ( 
      SELECT MAX(date) 
      FROM station_status 
      WHERE stationcode = i.stationcode 
    )
"""
stations = []
with cursorContext(Db.db) as cursor:
    try:
        cursor.execute(query, ())
    except Exception as e:
        print(Fore.RED + "Erreur lors de récupération des données : " + str(e) + Style.RESET_ALL)

    stations = cursor.fetchall()

data = pd.DataFrame(stations, columns=cursor.column_names)
display(data.head())

NameError: name 'Db' is not defined

In [ ]:
from folium.features import DivIcon

def generate_icon(perc, text, color):
    return DivIcon(
        html=f"""
        <svg width="50px" height="50px" viewBox="0 0 42 42" className="donut" aria-labelledby="beers-title beers-desc" role="img">
                <circle className="donut-hole" cx="21" cy="21" r="15.91549430918954" fill="white" role="presentation"></circle>
                <circle className="donut-ring" cx="21" cy="21" r="15.91549430918954" fill="transparent" stroke="#d2d3d4" strokeWidth="3" role="presentation"></circle>
                <circle className="donut-segment" cx="21" cy="21" r="15.91549430918954" fill="transparent" stroke="{color}" strokeWidth="3" strokeDasharray="{perc} {100 - perc}" strokeDashoffset="25" aria-labelledby="donut-segment-1-title donut-segment-1-desc">
                </circle>
                <g className="chart-text">
                    <text className="chart-number" x="50%" y="52%" dominantBaseline="middle" textAnchor="middle">
                        {text}
                    </text>
                </g>
            </svg>
        """,
        icon_size=(50, 50),
        popup_anchor=(0, -30)
    )

In [ ]:
import folium
from folium.plugins import MarkerCluster

print(len(data))

# filtrer donnée seulement paris
filter_data = data.loc[data['nom_arrondissement_communes'].str.lower() == 'paris']


m = folium.Map(location=[48.856, 2.352], zoom_start=12)
	
marker_cluster = MarkerCluster().add_to(m)


for index, row in filter_data.iterrows():
    pourc = round((row['numbikesavailable'] / row['capacity']) * 100) if row['capacity'] > 0 else 0;
    folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=generate_icon(perc=pourc, text=row['numbikesavailable'], color="hsl(130, 100%, 40%)")).add_to(marker_cluster)

m

In [5]:
from mysql.connector import errorcode
from colors import Fore, Style
from cursorContext import cursorContext
import pandas as pd

query = """
WITH cte_station_status_22h AS (
    SELECT
        ss.stationcode,
        si.name,
        si.coordonnees_geo,
        AVG(ss.numbikesavailable / si.capacity) AS remplissage_moyen
    FROM
        station_status AS ss
    INNER JOIN
        station_information AS si ON ss.stationcode = si.stationcode
    WHERE
        HOUR(ss.date) = 22 AND si.capacity > 0
    GROUP BY
         si.stationcode, si.name
),
cte_station_vide AS (
    SELECT stationcode, name, remplissage_moyen, coordonnees_geo
    FROM cte_station_status_22h
    WHERE remplissage_moyen < 0.1
),
cte_station_proche AS (
    SELECT
        stv.stationcode,
        stv.name,
        stv.remplissage_moyen,
        sp.stationcode AS station_pleine,
        sp.name AS station_pleine_name,
        sp.remplissage_moyen AS remplissage_station_pleine,
        ST_Distance_Sphere(stv.coordonnees_geo, sp.coordonnees_geo) AS distance,
        ROW_NUMBER() OVER (PARTITION BY stv.stationcode, stv.name ORDER BY ST_Distance_Sphere(stv.coordonnees_geo, sp.coordonnees_geo)) AS row_num    FROM
        cte_station_vide AS stv
    LEFT JOIN cte_station_status_22h AS sp ON sp.remplissage_moyen >= 0.8
)
SELECT
    stationcode,
    name,
    remplissage_moyen,
    station_pleine,
    station_pleine_name,
    remplissage_station_pleine,
    distance
FROM
    cte_station_proche
WHERE
    row_num = 1;
"""

stations = []
with cursorContext(Db.db) as cursor:
    try:
        cursor.execute(query, ())
    except Exception as e:
        print(Fore.RED + "Erreur lors de récupération des données : " + str(e) + Style.RESET_ALL)

    stations = cursor.fetchall()

data = pd.DataFrame(stations, columns=cursor.column_names)
display(data)

,stationcode,name,remplissage_moyen,station_pleine,station_pleine_name,remplissage_station_pleine,distance
0,10023,Gare de l'Est - Verdun,0.06382021,11003,Keller - La Roquette,0.83370798,2520.580329
1,10033,Gare du Nord - Saint-Vincent de Paul,0.09929255,11003,Keller - La Roquette,0.83370798,3282.174667
2,10107,Gare du Nord - Hôpital Lariboisière,0.09513032,11003,Keller - La Roquette,0.83370798,3381.008817
3,11014,Place de la Nation - Taillebourg,0.01426729,12027,Montgallet - Charenton,0.84375000,1067.078385
4,12164,Stade Pershing,0E-8,43402,Place de Verdun,0.84022606,1495.857859
...,...,...,...,...,...,...,...
71,9022,Rossini - Laffitte,0.08558457,11003,Keller - La Roquette,0.83370798,3370.263630
72,9023,Laffitte - Italiens,0.09678910,11003,Keller - La Roquette,0.83370798,3320.942114
73,9035,Gare Saint-Lazare - Place de Budapest,0.08801144,15025,Dupleix - du Guesclin,0.81653590,3681.723069
74,9102,Londres - Place d'Estienne d'Orves,0.09287234,15025,Dupleix - du Guesclin,0.81653590,3739.462461


In [10]:
from tableDefinition import TABLES, build_table_query, log_table

for table in TABLES:
    print(build_table_query(table.name, table.columns, table.foreign_keys, table.bonuses))


print(build_table_query(log_table["name"], log_table["columns"]))


CREATE TABLE station_information (stationcode VARCHAR(20) NOT NULL PRIMARY KEY, name VARCHAR(255) NOT NULL, capacity INT NOT NULL, coordonnees_geo POINT NOT NULL, nom_arrondissement_communes VARCHAR(255) NOT NULL);
CREATE TABLE station_status (date DATETIME NOT NULL, stationcode VARCHAR(20) NOT NULL, is_installed VARCHAR(3) NOT NULL, numdocksavailable INT NOT NULL, numbikesavailable INT NOT NULL, mechanical INT NOT NULL, ebike INT NOT NULL, CONSTRAINT `fk_stationcode` FOREIGN KEY (stationcode) REFERENCES station_information (stationcode) ON DELETE CASCADE ON UPDATE CASCADE, PRIMARY KEY (stationcode, date));
CREATE TABLE history_change (DATE DATETIME NOT NULL, USER VARCHAR(50) NOT NULL, TYPE VARCHAR(50) NOT NULL, TABLE_AFFECTED VARCHAR(50) NOT NULL, MSG VARCHAR(255) NOT NULL);
